# 走行制御 
2020/4/23 ver.3 ドリフト追加  
2020/4/23 ver.2 低中速モード  
2020/4/21 ver.1 基本走行  

In [1]:
TASK = 'c5-202a'

In [2]:
from jetcam.csi_camera import CSICamera
#camera = CSICamera(width=224, height=224, capture_fps=65)
camera = CSICamera(width=224, height=224)

#from jetcam.usb_camera import USBCamera
#camera = USBCamera(width=224, height=224)

In [3]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Load the optimized model by executing the cell below

In [4]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(TASK+'_trt.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Create the racecar class

In [5]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,
* xy出力値は-1...1
* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [6]:
import ipywidgets

speed_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.05,value=0.0, description='speed')
display(speed_slider)

FloatSlider(value=0.0, description='speed', max=1.0, step=0.05)

In [ ]:
from utils import preprocess
import numpy as np
import time

STEERING_BIAS = -0.15
STEERING_GAIN = -0.45

THROTTLE_BIAS = 0.08
THROTTLE_GAIN = -0.14

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    
    car.throttle = speed_slider.value * THROTTLE_GAIN + THROTTLE_BIAS
    car.steering = x * STEERING_GAIN + STEERING_BIAS

### カメラ停止

In [ ]:
camera.unobserve_all()